Постановка задачи распределения.
Цель: распределить задачи между участниками так, чтобы:
1. Каждая задачи была назначена ровно одному участнику.
2. Суммарная нагрузка по времени у участников была максимально сбалансирована.
3. Учитывались предпочтения участников (максимизировать суммарное удовлетворение).

Математическая модель:\nПеременные:\
$ x_{ij} \in {0,1}$ - назначена ли задача i участнику j.\
$j \in {p1, p2, p3}, i \in {T3.1,...T3.11}$.\
Параметры:\
$t_i$ - трудомкость задачи i (мат ожидание из таблицы).\
$p_{ij}$ - предпочтение участника j по задаче i.\
Ограничения:
1. Каждая задача выполняется ровно одним участником
$$\sum_j x_{ij}=1 \forall i$$
2. Балансировка нагрузки:\
Введём переменную $L_max$ - максимальная нагрузка среди участников.
$$\sum_i t_i * x_{ij} \leq L_{max} \forall j$$
Минимизируем $L_{max}$.\
Целевая функция:\
Максимизировать суммарное взвешенное удовлетворение с учётом баланса:
$$max(\alpha * \sum_{i,j}p_{ij}*x_{ij} - \beta * L_{max})$$
где $\alpha$ и $\beta$ - веса.

In [1]:
from pulp import *

# Данные
tasks = ['T3.1', 'T3.2', 'T3.3', 'T3.4', 'T3.5', 'T3.6', 'T3.7', 'T3.8', 'T3.9', 'T3.10', 'T3.11']
members = ['Участник 1', 'Участник 2', 'Участник 3']

# Трудозатраты (мат. ожидание)
time = {
    'T3.1': 6.0, 'T3.2': 2.0, 'T3.3': 3.2, 'T3.4': 2.0, 'T3.5': 3.0,
    'T3.6': 2.0, 'T3.7': 2.0, 'T3.8': 1.2, 'T3.9': 3.0, 'T3.10': 3.2, 'T3.11': 4.0
}

# Предпочтения (словарь словарей)
pref = {
    'Участник 1': {'T3.1': 4, 'T3.2': 2, 'T3.3': 3, 'T3.4': 2, 'T3.5': 3, 'T3.6': 5, 'T3.7': 5, 'T3.8': 4, 'T3.9': 5, 'T3.10': 5, 'T3.11': 4},
    'Участник 2':   {'T3.1': 2, 'T3.2': 5, 'T3.3': 5, 'T3.4': 4, 'T3.5': 5, 'T3.6': 3, 'T3.7': 3, 'T3.8': 3, 'T3.9': 4, 'T3.10': 4, 'T3.11': 3},
    'Участник 3':{'T3.1': 5, 'T3.2': 3, 'T3.3': 4, 'T3.4': 3, 'T3.5': 4, 'T3.6': 4, 'T3.7': 5, 'T3.8': 5, 'T3.9': 5, 'T3.10': 5, 'T3.11': 5}
}

# Создаём задачу
prob = LpProblem("Task_Assignment", LpMaximize)

# Переменные
x = LpVariable.dicts("x", (tasks, members), lowBound=0, upBound=1, cat='Binary')
L_max = LpVariable("L_max", lowBound=0)

# Целевая функция: максимизация предпочтений с балансировкой нагрузки
alpha = 1
beta = 10
prob += alpha * lpSum(pref[m][t] * x[t][m] for t in tasks for m in members) - beta * L_max

# Ограничения
# 1. Каждая задача назначается ровно одному участнику
for t in tasks:
    prob += lpSum(x[t][m] for m in members) == 1

# 2. L_max >= суммарной нагрузки каждого участника
for m in members:
    prob += lpSum(time[t] * x[t][m] for t in tasks) <= L_max

# Решаем
prob.solve(PULP_CBC_CMD(msg=False))

# Вывод результатов
print("Статус решения:", LpStatus[prob.status])
print("\nРаспределение задач:")
for m in members:
    print(f"\n{m}:")
    total_time = 0
    for t in tasks:
        if value(x[t][m]) == 1:
            print(f"  - {t} (время: {time[t]} ч, предпочтение: {pref[m][t]})")
            total_time += time[t]
    print(f"  Итого времени: {round(total_time, 1)} ч")

print(f"\nМаксимальная нагрузка (L_max): {value(L_max)} ч")
print(f"Суммарное удовлетворение (без весов): {value(alpha * lpSum(pref[m][t] * x[t][m] for t in tasks for m in members))}")

Статус решения: Optimal

Распределение задач:

Участник 1:
  - T3.6 (время: 2.0 ч, предпочтение: 5)
  - T3.7 (время: 2.0 ч, предпочтение: 5)
  - T3.9 (время: 3.0 ч, предпочтение: 5)
  - T3.11 (время: 4.0 ч, предпочтение: 4)
  Итого времени: 11.0 ч

Участник 2:
  - T3.2 (время: 2.0 ч, предпочтение: 5)
  - T3.3 (время: 3.2 ч, предпочтение: 5)
  - T3.4 (время: 2.0 ч, предпочтение: 4)
  - T3.5 (время: 3.0 ч, предпочтение: 5)
  Итого времени: 10.2 ч

Участник 3:
  - T3.1 (время: 6.0 ч, предпочтение: 5)
  - T3.8 (время: 1.2 ч, предпочтение: 5)
  - T3.10 (время: 3.2 ч, предпочтение: 5)
  Итого времени: 10.4 ч

Максимальная нагрузка (L_max): 11.0 ч
Суммарное удовлетворение (без весов): 53.0
